# Amazon OpenSearch Service에 Model(Remote Connector) 배포하기

패키지를 설치합니다.

In [ ]:
%pip install -U --quiet opensearch-py requests
%pip install -q boto3
%pip install -q requests
%pip install -q requests-aws4auth
%pip install -q opensearch-py
%pip install -q tqdm
%pip install -q boto3

SageMaker Notebook에 Attach된 IAM Role을 확인합니다. 중요한 것은 코드를 실행할 때 Assume된 Role이 아닌 실제 SageMaker에 Attach된 IAM Role을 확인해야 한다는 것입니다. 

In [ ]:
import boto3
import botocore

# Create a session with boto3
session = boto3.Session()

# Get the STS client
sts_client = session.client("sts")

# Get the caller identity
caller_identity = sts_client.get_caller_identity()

# Get the assumed role ARN
assumed_role_arn = caller_identity["Arn"]

# Get the role name from the assumed role ARN
role_name = assumed_role_arn.split("/")[1]

# Get the IAM client
iam_client = session.client("iam")

is_user = False

# Get the role details
try:
    role = iam_client.get_role(RoleName=role_name)
    iam_role_arn = role["Role"]["Arn"]

    print(f"The IAM role assumed now is: {assumed_role_arn}")
    print(f"The IAM role attached to this SageMaker notebook instance is: {iam_role_arn}")
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Opensearch에 연결할 정보를 설정합니다.

In [ ]:
session = boto3.Session()
region_name = session.region_name


def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs


import boto3, json

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]
aos_host

Opensearch에 연결합니다.

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import json

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=600,
)

# IAM Role에 OpenSearch의 ml_full_access 권한 부여하기

OpenSearch에 Model을 배포하기 위해서는 ml_full_access 권한이 필요합니다. 이를 위해서는 ml_full_access Role에 모델 배포 요청 Request를 Sign할 IAM Role ARN을 맵핑해주어야 합니다. 

먼저 ml_full_access의 Role Mapping을 확인합니다.

In [ ]:
import requests

security_url = "https://" + aos_host + "/_plugins/_security/api/rolesmapping/ml_full_access"

auth = (aos_credentials["username"], aos_credentials["password"])

create_role_mapping_body = {"users": [iam_role_arn]}

response = requests.get(
    security_url,
    auth=auth,
    # json=connector_payload,
)

json.loads(response.text)

현재 오직 `master` 사용자만이 ml_full_access 권한을 가지고 있을 것입니다. 여기에 `iam_role_arn`에 저장된 현재 IAM Role ARN을 추가해줍니다. 

In [ ]:
if is_user is True:
    # User ARN일 경우
    create_role_mapping_body = {"users": ["master", iam_role_arn]}
else:
    # IAM Role ARN일 경우
    create_role_mapping_body = {
        "users": ["master"],
        "backend_roles": [iam_role_arn],
    }

response = requests.put(
    security_url,
    auth=auth,
    json=create_role_mapping_body,
)

json.loads(response.text)

Role Mapping이 잘 되었는지 확인합니다.

In [ ]:
response = requests.get(
    security_url,
    auth=auth,
    # json=connector_payload,
)

json.loads(response.text)

모델 커넥터를 생성하는 요청을 보낼 때는 커넥터의 권한을 OpenSearch에게 넘겨줄 수 있는 PassRole 권한이 필요합니다. 하지만 OpenSearch의 Internal User(master)는 IAM 사용자가 아니므로 해당 권한을 설정할 수 없습니다. 따라서 credentials 정보를 `iam_role_arn`에 저장된 IAM Role의 정보로 변경합니다.

# 임베딩 원격 모델 배포하기

In [ ]:
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region=region_name)

모델 커넥터 생성 요청을 위한 Base URL을 설정합니다. 

In [ ]:
base_url = "https://" + aos_host + "/_plugins/_ml"
base_url

원격 모델 커넥터를 생성합니다.

In [ ]:
import requests

# Bedrock Titan Embeddings G1 - Text Connector 생성

connector_payload = {
    "name": "Amazon Bedrock Connector: embedding",
    "description": "The connector to bedrock Titan embedding model",
    "version": 1,
    "protocol": "aws_sigv4",
    "parameters": {
        "region": region_name,
        "service_name": "bedrock",
        "model": "amazon.titan-embed-text-v2:0",
    },
    "credential": {"roleArn": iam_role_arn},
    "actions": [
        {
            "action_type": "predict",
            "method": "POST",
            "url": "https://bedrock-runtime.${parameters.region}.amazonaws.com/model/${parameters.model}/invoke",
            "headers": {"content-type": "application/json", "x-amz-content-sha256": "required"},
            "request_body": '{ "inputText": "${parameters.inputText}" }',
            "pre_process_function": "connector.pre_process.bedrock.embedding",
            "post_process_function": "connector.post_process.bedrock.embedding",
        }
    ],
}

# Send the request
response = requests.post(
    base_url + "/connectors/_create",
    auth=auth,
    json=connector_payload,
)

# Print the response
print(response.text)

In [ ]:
obj = json.loads(response.text)
connector_id = obj["connector_id"]
connector_id

원격 모델을 위한 모델 그룹을 생성합니다.

In [ ]:
register_model_group_payload = {
    "name": "Bedrock Models for titan text v2",
    "description": "This is a model group for Amazon Bedrock remote model",
}

# Send the request
response = requests.post(
    base_url + "/model_groups/_register",
    auth=auth,
    json=register_model_group_payload,
)

# Print the response
print(response.text)

In [ ]:
obj = json.loads(response.text)
model_group_id = obj["model_group_id"]
model_group_id

앞서 생성한 커넥터와 모델 그룹을 사용하여 모델을 등록합니다.

In [ ]:
register_model_payload = {
    "name": "Bedrock text embedding model",
    "function_name": "remote",
    "model_group_id": model_group_id,
    "description": "This is Bedrock Titan Embeddings V2 - Text model",
    "connector_id": connector_id,
}

# Send the request
response = requests.post(
    base_url + "/models/_register",
    auth=auth,
    json=register_model_payload,
)

# Print the response
print(response.text)

In [ ]:
obj = json.loads(response.text)
model_id = obj["model_id"]
print(model_id)

앞서 등록한 모델을 배포합니다.

In [ ]:
deploy_model_payload = {}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_deploy",
    auth=auth,
    json=deploy_model_payload,
)

# Print the response
print(response.text)

모델이 잘 배포되었는지 테스트해봅니다.

In [ ]:
predict_model_payload = {"parameters": {"inputText": "hello goodbye"}}

# Send the request
response = requests.post(
    base_url + "/models/" + model_id + "/_predict",
    auth=auth,
    json=predict_model_payload,
)

# Print the response
len(response.json()["inference_results"][0]["output"][0]["data"])

이후 테스트에 사용하기 위해 모델 아이디를 저장합니다.

In [ ]:
%store model_id